# İş Problemi

Facebook kısa süre önce mevcut "maximumbidding" adı verilen teklif verme türüne alternatif olarak yeni bir teklif türü olan "average bidding"’i tanıttı. 

Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test etmeye karar verdi ve average bidding'inmaximumbidding'den daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B testi yapmak istiyor.

A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden bu A/B testinin sonuçlarını analiz etmenizi bekliyor. Bombabomba.com için nihai başarı ölçütü Purchase'dır. Bu nedenle, istatistiksel testler için Purchase metriğine odaklanılmalıdır.

# Veri Seti Hikayesi

Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra
buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri
ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average
Bidding uygulanmıştır.

# Görev 1: Veriyi Hazırlama ve Analiz Etme

Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı
değişkenlere atayınız.

Adım 2: Kontrol ve test grubu verilerini analiz ediniz.

Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import shapiro, levene, ttest_ind

control = pd.read_excel('/Users/mericgenc/Desktop/Python - Data Science/Miuul Data Scientist Path/03 - Measurement Problems/05_case_study_I_ab_testing/ab_testing.xlsx', sheet_name='Control Group')
test = pd.read_excel('/Users/mericgenc/Desktop/Python - Data Science/Miuul Data Scientist Path/03 - Measurement Problems/05_case_study_I_ab_testing/ab_testing.xlsx', sheet_name='Test Group')

In [2]:
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [3]:
control.describe([0.01,0.25,0.5,0.75,0.99]).T

,count,mean,std,min,1%,25%,50%,75%,99%,max
Impression,40.0,101711.449068,20302.157862,45475.942965,58072.042430,85726.690349,99790.701078,115212.816543,143105.791105,147539.336329
Click,40.0,5100.657373,1329.985498,2189.753157,2535.120793,4124.304129,5001.220602,5923.803596,7761.795112,7959.125069
Purchase,40.0,550.894059,134.108201,267.028943,285.770764,470.095533,531.206307,637.957088,790.187785,801.795020
Earning,40.0,1908.568300,302.917783,1253.989525,1267.764100,1685.847205,1975.160522,2119.802784,2481.308737,2497.295218


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [5]:
test.describe([0.01,0.25,0.5,0.75,0.99]).T

,count,mean,std,min,1%,25%,50%,75%,99%,max
Impression,40.0,120512.411758,18807.448712,79033.834921,79112.254954,112691.970770,119291.300775,132050.578933,158245.263797,158605.920483
Click,40.0,3967.549761,923.095073,1836.629861,2005.233567,3376.819024,3931.359804,4660.497911,6012.877296,6019.695079
Purchase,40.0,582.106097,161.152513,311.629515,325.214426,444.626828,551.355732,699.862360,876.576104,889.910460
Earning,40.0,2514.890733,282.730852,1939.611243,1942.784238,2280.537426,2544.666107,2761.545405,3091.940894,3171.489708


In [6]:
test['group'] = 'test'
control['group'] = 'control'

df = pd.concat([control,test],axis=0, ignore_index=True)
df

,Impression,Click,Purchase,Earning,group
0,82529.459271,6090.077317,665.211255,2311.277143,control
1,98050.451926,3382.861786,315.084895,1742.806855,control
2,82696.023549,4167.965750,458.083738,1797.827447,control
3,109914.400398,4910.882240,487.090773,1696.229178,control
4,108457.762630,5987.655811,441.034050,1543.720179,control
...,...,...,...,...,...
75,79234.911929,6002.213585,382.047116,2277.863984,test
76,130702.239410,3626.320072,449.824592,2530.841327,test
77,116481.873365,4702.782468,472.453725,2597.917632,test
78,79033.834921,4495.428177,425.359102,2595.857880,test


# Görev 2: A/B Testinin Hipotezinin Tanımlanması

Adım 1: Hipotezi tanımlayınız.

H0 : M1 = M2

H1 : M1!= M2

Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.

In [7]:
# H0: M1 = M2 (Max Bidding ile Avg Bidding ortalamaları arasında istatistiki anlamda bir fark yoktur)
# H1: M1 != M2 (Max Bidding ile Avg Bidding ortalamaları arasında istatistiki anlamda bir fark vardır)

df.groupby('group').agg({'Purchase':'mean'})

,Purchase
group,
control,550.894059
test,582.106097


In [8]:
# Test Grubunun purchase ortalaması daha iyi gözüküyor. Bu tesadüfen mi oluşmuş öğrenmek için hipotez testi yapacağız.

# Görev 3: Hipotez Testinin Gerçekleştirilmesi

Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.

Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni
üzerinden ayrı ayrı test ediniz.

    Normallik Varsayımı :
    H0: Normal dağılım varsayımı sağlanmaktadır.
    H1: Normal dağılım varsayımı sağlanmamaktadır.
    p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ

Test sonucuna göre normallik varsayımı kontrol ve test grupları için sağlanıyor mu ? Elde edilen p-value değerlerini yorumlayınız.

    Varyans Homojenliği :
    H0: Varyanslar homojendir.
    H1: Varyanslar homojen Değildir.
    p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
    Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını Purchase değişkeni üzerinden test ediniz.
    Test sonucuna göre normallik varsayımı sağlanıyor mu? Elde edilen p-value değerlerini yorumlayınız.

Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiki
olarak anlamlı bir fark olup olmadığını yorumlayınız.

In [9]:
test_stat, pvalue = shapiro(df.loc[df['group']=='control', 'Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [10]:
test_stat, pvalue = shapiro(df.loc[df['group']=='test', 'Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


In [11]:
# Kontrol Grubunun da Test Grubunun da p-value'su 0.05'in üzerindedir. Bu durumda H0'ı reddedemiyoruz. Normallik varsayımı sağlanmaktadır.

In [12]:
# Varyans Homojenliği;

test_stat, pvalue = levene(df.loc[df['group']=='test', 'Purchase'],
                           df.loc[df['group']=='control', 'Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


In [13]:
# p-value 0.05'ten büyük olduğu için H0'ı reddedemiyoruz. Yani varyans homojendir.
# İki varsayım da sağlandığı için bağımsız iki örneklem t testi (parametrik test) yapacağız

In [14]:
test_stat, pvalue = ttest_ind(test['Purchase'],
                              control['Purchase'],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9416, p-value = 0.3493


In [15]:
# p_value 0.05'ten büyük çıktı. Yani H0'ı reddedemiyoruz. Bu durumda kontrol ve test grubu purchase ortalamaları arasında istatistiki olarak anlamlı bir fark yoktur.

# Görev 4: Sonuçların Analizi

Adım 1: Hangi testi kullandınız, sebeplerini belirtiniz.

Adım 2: Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz.

Önce Normallik Varsayımı testi yapılmıştır. İki grup da normal dağılımda olduğu için Varyans Homojenliğine baktık. Bu da sağlandığı için Bağımsız İki Örneklem T Testi yani Parametric Test yaptık.

Test sonuçlarına göre iki grup arasında anlamlı bir fark olmadığını bulduk. Bu yüzden müşterinin bir değişiklik yapması zorunlu değildir. İstediği yöntem ile devam edebilir.
Ancak buradaki diğer istatistiklerdeki farklılıklara da bakılmalıdır. Tıklanma, Etkileşim, Kazanç ve Dönüşüm Oranlarındaki farklılıklara göre hangi yöntemin daha kazançlı olduğuna karar verilebilir.
Tıklama başına para ödendiği için hangi yöntemde tıklama oranının daha düşük olduğu tespit edilip Click Through Rate oranına bakılır.
İki grup gözlenmeye devam edilir.